# Assignment 1 - Backpropagation

### Notebook created by Anirudh Swaminathan from ECE department majoring in Intelligent Systems, Robotics and Control for the course ECE285 Machine Learning for Image Processing for Fall 2019

## 2. Getting Started

In [ ]:
import numpy as np
from matplotlib import pyplot

## 3. Read MNIST Data

In [ ]:
import MNISTtools
help(MNISTtools.load)
help(MNISTtools.show)

#### Question 1

In [ ]:
# Load the data
xtrain, ltrain = MNISTtools.load(path='./datasets/MNIST')
print(xtrain.shape)
print(ltrain.shape)

The shape of $xtrain$ is $(784, 60000)$<br>
The shape of $ltrain$ is $(60000, )$<br>
The size of the training set, i.e., the number of images in the training set is $60000$<br>
The feature dimension is $784$

#### Question 2

In [ ]:
# Displaying the image at index 42
MNISTtools.show(xtrain[:, 42])

# Print its corresponding label
print(ltrain[42])

The image at the index $42$ has been displayed.<br>
The corresponding label has been printed and is found to be $7$

#### Question 3

In [ ]:
# Find the range and type of xtrain
min_x = np.amin(xtrain)
max_x = np.amax(xtrain)

print("Range of xtrain is from ", min_x, " to ", max_x)
print("Data type of xtrain is ", xtrain.dtype)

The range of values for $xtrain$ is from $0$ to $255$<br>
The type of $xtrain$ is $uint8$

#### Question 4

In [ ]:
def normalize_MNIST_images(x):
    # Convert the uint8 input into float32 for ease of normalization
    fl_x = x.astype(np.float32)
    
    # Normalize [0 to 255] to [-1 to 1]
    # This means mapping 0 to -1, 255 to 1, and 127.5 to 0
    ret = 2*(fl_x - 255/2.0) / 255
    return ret

In [ ]:
norm_x_train = normalize_MNIST_images(xtrain)
print(norm_x_train.shape)
print("Range of normalized xtrain is", np.amin(norm_x_train), "to", np.amax(norm_x_train))
print("Data type of normalized xtrain is", norm_x_train.dtype)

We wrote the function to normalize the training data from $[0 to 255]$ to $[-1 to 1]$<br>
We converted $xtrain$ which was of type $uint8$ into a vector of type $float32$<br>
We then mapped $0$ to $-1$, $255$ to $1$ by subtracting the mid, which is $127.5$ and then dividing by mid, which is $127.5$<br>
We then stored the normalized $xtrain$ in the variable $norm\_x\_train$

#### Question 5

In [ ]:
# Complete the code below
def label2onehot(lbl):
    # Creates a placeholder of size (10, 60000)
    d = np.zeros((lbl.max() + 1, lbl.size))
    
    # One-hot encode the labels
    d[lbl, np.arange(lbl.size)] = 1
    return d

In [ ]:
dtrain = label2onehot(ltrain)
print(dtrain.shape)
print(np.amin(dtrain), np.amax(dtrain))
print("Label at index 42 is", ltrain[42])
print("Corresponding one-hot encodiing is", dtrain[:, 42])

The one hot encoding line works as the $1^{st}$ index is traveresed independently of the $2^{nd}$ index<br>
So, for each image given by the $2^{nd}$ axis, only the row index given by the value of the label is assigned $1$<br>
Thus, $0$ maps to $[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]$ and 9 maps to $[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]$<br><br>
We also checked the label for image $42$. The label is $7$ and the corresponding one-hot encoding is $[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]$

#### Question 6

In [ ]:
def onehot2label(d):
    lbl = d.argmax(axis=0)
    return lbl

In [ ]:
# Checking if this works
lab = dtrain[:, 42]
che = onehot2label(lab)

print("One-hot answer", che, "| Original:", ltrain[42])
assert(che == ltrain[42])

We have thus checked if our implementation of recovering the label from one-hot encoding is correct<br>
The label of the image at index at $42$ is $7$<br>
The $onehot2label()$ function recovers this correctly

## 4. Activation Functions

#### Question 7

In [ ]:
# Implement the softmax function
def softmax(a):
    # Calculate the max value
    M = np.max(a, axis=0)
    
    # Subtract for easier exponential calculation
    a_m = a - M
    
    # Calculate the exponent for each class for each image
    exp_a_m = np.exp(a_m)
    
    # Calculate the sum for each class
    den = np.sum(exp_a_m, axis=0)
    
    # Get the probabilities for each class for each image
    g_a = exp_a_m / den
    return g_a

#### Question 8

We need to show that $\frac{\partial{g(a)_i}}{\partial{a_i}} = g(a)_i(1 - g(a)_i)$<br><br>
By definition above, Softmax is $$y_i = g(a)_i = \frac{exp(a_i)}{\sum_{j=1}^{10}exp(a_j)} $$
So, $$ \frac{\partial{g(a)_i}}{\partial{a_i}} = \frac{\partial \left({\frac{exp(a_i)}{\sum_{j=1}^{10}exp(a_j)}} \right)}{\partial{a_i}} $$
Using the division rule of derivatives, we have $$ \frac{\partial{g(a)_i}}{\partial{a_i}} = \frac{\sum_{j=1}^{10}exp(a_j)\frac{\partial{exp(a_i)}}{\partial{a_i}} - exp(a_i)\frac{\partial \left( {\sum_{j=1}^{10}exp(a_j)} \right)}{\partial{a_i}}}{\left( \sum_{j=1}^{10}exp(a_j) \right)^2} $$